In [1]:
import numpy as np
import scipy.io
from scipy import signal
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    recall_score,
    f1_score,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGNetv4
import copy
import random

import pandas as pd


torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
c:\Users\machi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
c:\Users\machi\anaconda3\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be ab

In [2]:
def plot_learning_curves(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = range(1, len(train_losses) + 1)

    # Curva de Perda
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Perda Treinamento")
    plt.plot(epochs, val_losses, label="Perda Validação")
    plt.xlabel("Épocas")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Curva de Perda: Treino e validação")

    # Curva de Acurácia
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Acurácia Treinamento")
    plt.plot(epochs, val_accuracies, label="Acurácia Validação")
    plt.xlabel("Épocas")
    plt.ylabel("Acurácia (%)")
    plt.legend()
    plt.title("Curva de Acurácia: Treino e validação")

    plt.tight_layout()
    plt.show()

In [3]:
def evaluate(model, test_loader, chanels):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.inference_mode():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Metrics
    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average="weighted")
    f1 = f1_score(all_labels, all_preds, average="weighted")

    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test set Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    classes = np.unique(np.concatenate((all_labels, all_preds)))

    # disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    # fig, ax = plt.subplots(figsize=(15, 15))  # aumenta a figura
    # disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")  # rotaciona os rótulos para caber melhor
    # plt.show()

    # print(all_labels)
    # print(all_preds)

    return accuracy, recall, f1

In [ ]:
def bandpass_filter(
    dados, taxa_amostragem, freq_corte_low, freq_corte_high, ordem_filtro
):
    """
    Filtra dados EEG utilizando um filtro Butterworth passa-banda.
    Parâmetros:
    dados (ndarray): Dados do EEG com formato (número de eletrodos, número de amostras, número de frequências, número de trials).
    taxa_amostragem (int): Frequência de amostragem dos sinais EEG (Hz).
    freq_corte_low (float): Frequência de corte inferior do filtro passa-banda (Hz).
    freq_corte_high (float): Frequência de corte superior do filtro passa-banda (Hz).
    ordem_filtro (int): Ordem do filtro Butterworth.

    Retorna:
    ndarray: Dados EEG filtrados.
    """

    # **Construção do filtro passa-banda**

    # Cria o filtro passa-banda com os parâmetros especificados

    b, a = signal.butter(
        ordem_filtro,
        [freq_corte_low, freq_corte_high],
        btype="bandpass",
        analog=False,
        output="ba",
        fs=taxa_amostragem,
    )

    # **Filtragem dos dados**
    # Realiza o processo de filtragem para todas as frequências, trials e eletrodos

    num_eletrodos, num_amostras, num_freqs, num_trials = dados.shape

    filtered_data = np.zeros_like(dados)
    # Filtra os dados para cada frequência, trial e eletrodo
    for f in range(num_freqs):  # Para cada frequência de estimulação
        for trial in range(num_trials):  # Para cada trial
            for eletrodo in range(num_eletrodos):  # Para cada eletrodo
                # Filtra o sinal com o filtro de fase zero
                eletrodo_filtrado = signal.filtfilt(b, a, dados[eletrodo, :, f, trial])
                # Substitui o dado original pelo filtrado
                filtered_data[eletrodo, :, f, trial] = eletrodo_filtrado

    return filtered_data

In [ ]:
def separar_em_janelas(matriz, tamanho_janela, incluir_ultimo=False):
    # Calcular o número total de linhas
    total_linhas = matriz.shape[0]

    # Calcular o número de grupos necessário
    num_janelas = total_linhas // tamanho_janela

    # Inicializar uma lista vazia para armazenar os grupos
    janelas = []

    # Iterar sobre a matriz, pegando as linhas de tamanho_grupo em tamanho_grupo e armazenando em grupos
    for i in range(0, total_linhas, tamanho_janela):
        janela = matriz[i : i + tamanho_janela]  # Pegar o grupo de tamanho_grupo linhas
        janelas.append(janela)  # Adicionar o grupo à lista

    # Se incluir_ultimo for False e houver linhas restantes, remover o último grupo
    if not incluir_ultimo and total_linhas % tamanho_janela != 0:
        janelas.pop()

    return janelas, num_janelas

In [15]:
def train(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    num_epochs=100,
    device=0,
    save_path="best_model_raw.pth",
):
    best_val_accuracy = 0.0
    model.to(device)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # eval train
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        train_accuracy = train_correct / train_total
        avg_train_loss = running_loss / len(train_loader)

        # eval validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.inference_mode():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # val accuracy
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        # Save if best vall acc
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), save_path)
            # print(f"Best model saved with accuracy: {best_val_accuracy:.4f}")

    #     print(
    #         f"Epoch {epoch + 1}/{num_epochs}: "
    #         f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
    #         f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
    #         )
    # plot_learning_curves(train_losses, val_losses, train_accuracies, val_accuracies)
    model.load_state_dict(best_model)
    return model

In [ ]:
frequencias_e_fases = scipy.io.loadmat(
    "C:/Users/machi/Documents/Mestrado/repos/data/benchmark/Freq_Phase.mat"
)
frequencias = frequencias_e_fases["freqs"]
frequencias = np.round(frequencias, 2).ravel()
fases = frequencias_e_fases["phases"]
filter_order = 10
freq_cut_high = 70
freq_cut_low = 6
sample_rate = 250
num_harmonica = 5
delay = 160  # 160 amostras, 0,5s (sem estimulação) + 0,14s (latencia para começo da evocação)
inform_fase = 0

In [ ]:
seed = 42
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

# Lista para armazenar as métricas de todos os usuários e sessões
metricas_usuarios = []

for user in range(1, 36):
    file_path = f"C:/Users/machi/Documents/Mestrado/repos/data/benchmark/S{user}.mat"
    print(f"Processando Usuário {user}")

    # Carregar e preparar dados
    data = scipy.io.loadmat(file_path)["data"]
    data = bandpass_filter(data, sample_rate, freq_cut_low, freq_cut_high, filter_order)
    data = data[:, (delay) : (delay + 1250), :, :]

    # Parâmetros de janelas e sessões
    tamanho_da_janela = 0.6
    tamanho_da_janela = int(np.ceil(tamanho_da_janela * sample_rate))
    inform_fase = 0

    # Eletrodos e frequências de interesse
    occipital_electrodes = np.array([47, 53, 54, 55, 56, 57, 60, 61, 62])
    frequencias_desejadas = frequencias[:8]
    indices = [np.where(frequencias == freq)[0][0] for freq in frequencias_desejadas]

    num_canais, num_amostras, num_freqs, num_sections = data.shape
    n_samps = num_amostras
    n_freqs_sel = len(indices)

    metricas_crossval = []

    # Leave-one-session-out cross-validation
    for sessao_teste in range(6):  # 6 sessões no total
        sessoes_treino = [s for s in range(6) if s != sessao_teste]
        sessoes_teste = [sessao_teste]

        x_treino = []
        rotulos_treino = []

        x_teste = []
        rotulos_teste = []

        for sessao in sessoes_treino:
            for freq in range(len(indices)):

                eeg_trial = data[occipital_electrodes, :, indices[freq], sessao]

                eeg_trial_janelas, numero_janelas = separar_em_janelas(
                    eeg_trial.T, tamanho_da_janela
                )
                eeg_trial_janelas_array = np.stack(eeg_trial_janelas)
                eeg_trial_janelas_array = eeg_trial_janelas_array.transpose(0, 2, 1)

                print(eeg_trial_janelas_array.shape)

                x_treino.append(eeg_trial_janelas_array)

                # Adiciona um rótulo 'freq' para cada janela gerada
                rotulos_treino.extend([frequencias[freq]] * numero_janelas)

        x_treino = np.concatenate(x_treino, axis=0)

        for sessao in sessoes_teste:
            for freq in range(len(indices)):

                eeg_trial = data[occipital_electrodes, :, indices[freq], sessao]

                eeg_trial_janelas, numero_janelas = separar_em_janelas(
                    eeg_trial.T, tamanho_da_janela
                )
                eeg_trial_janelas_array = np.stack(eeg_trial_janelas)
                eeg_trial_janelas_array = eeg_trial_janelas_array.transpose(0, 2, 1)

                print(eeg_trial_janelas_array.shape)

                x_teste.append(eeg_trial_janelas_array)

                # Adiciona um rótulo 'freq' para cada janela gerada
                rotulos_teste.extend([frequencias[freq]] * numero_janelas)

        x_teste = np.concatenate(x_teste, axis=0)

        # Mapeamento de rótulos
        mapeamento = {
            rotulo: i for i, rotulo in enumerate(sorted(frequencias_desejadas))
        }
        rotulos_treinamento = torch.tensor(
            [mapeamento[rotulo.item()] for rotulo in rotulos_treino]
        )
        rotulos_teste = torch.tensor(
            [mapeamento[rotulo.item()] for rotulo in rotulos_teste]
        )

        # Converter para tensores
        X_treino = torch.from_numpy(x_treino.copy()).float().to(device)
        X_teste = torch.from_numpy(x_teste.copy()).float().to(device)
        Y_treino = rotulos_treinamento.to(torch.long).to(device)
        Y_teste = rotulos_teste.to(torch.long).to(device)

        print("X_treino:", X_treino.shape)
        print("X_teste:", X_teste.shape)
        print("Y_treino:", Y_treino.shape)
        print("Y_teste:", Y_teste.shape)

        np.save(f"bruto_subj_{user}_session_{sessao_teste}_train.npy", x_treino)
        np.save(
            f"bruto_subj_{user}_session_{sessao_teste}_labels_train.npy", Y_treino.cpu()
        )
        np.save(f"bruto_subj_{user}_session_{sessao_teste}_test.npy", x_teste)
        np.save(
            f"bruto_subj_{user}_session_{sessao_teste}_labels_test.npy", Y_teste.cpu()
        )

        # Configurar modelo e treinamento
        model = EEGNetv4(
            n_chans=9,
            n_outputs=40,
            n_times=tamanho_da_janela,
            kernel_length=(tamanho_da_janela // 2),
        )
        model = model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0001)

        dataset = TensorDataset(X_treino, Y_treino)
        train_size = int(0.85 * len(dataset))
        val_size = len(dataset) - train_size

        train_dataset, val_dataset = random_split(
            dataset,
            [train_size, val_size],
            generator=torch.Generator().manual_seed(seed),
        )

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
        test_loader = DataLoader(
            TensorDataset(X_teste, Y_teste), batch_size=10, shuffle=False
        )

        # Treinar
        best_model = train(
            model, train_loader, val_loader, criterion, optimizer, num_epochs=1000
        )

        # Avaliar
        accuracy, recall, f1 = evaluate(
            best_model, test_loader, chanels=len(frequencias_desejadas)
        )

        # Armazenar métricas
        metricas_crossval.append(
            {
                "usuario": user,
                "sessao_teste": sessao_teste,
                "acuracia": accuracy,
                "recall": recall,
                "f1-score": f1,
            }
        )

        print(
            f"Usuário {user} - Sessão {sessao_teste} Finalizada: Acurácia={accuracy:.4f}, Recall={recall:.4f}, F1={f1:.4f}"
        )

    # Salvar as métricas de cada usuário
    metricas_usuarios.extend(metricas_crossval)

    print("-" * 50)

# =====================================
# Salvar resultados finais em CSV
# =====================================

# Criar um DataFrame com as métricas
df_metricas = pd.DataFrame(metricas_usuarios)

# Salvar como CSV
df_metricas.to_csv("metricas_treinamento_06_segundos_dados_brutos.csv", index=False)

Processando Usuário 1
